In [2]:
import requests


def download_file_with_custom_headers(url, local_filename):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()  # This will throw an exception for non-200 responses
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print("File downloaded successfully.")


# Example usage
file_url = 'https://raw.githubusercontent.com/toperkov/RF-FESB-2023-24/main/lab3/Download/setupapi.dev2.log'
output_dir = 'setupapi.dev2.log'
download_file_with_custom_headers(file_url, output_dir)

File downloaded successfully.


In [22]:
import re
import datetime
import pandas as pd  # Corrected import statement for pandas

usb_devices = []

log_path = 'setupapi.dev2.log'

# Read the contents of the setupapi.dev.log file
with open(log_path, "r") as log_file:
     # Store information about each USB device in a list of dictionaries
     for line in log_file:
         # Find all USB device installation events and extract information about each device
         match = re.match(r'^>>>  \[Device Install.*#(Disk&Ven_[A-Za-z0-9]+)&(Prod_([\w\s\S]+?))&(Rev_([\w\s\S]+?))#([\w\s\S]+?)#.*\]', line)
         if not match:
             continue

         usb_devices.append({
             "device_vendor_id": match.group(1),
             "device_product_id": match.group(2),
             "device_instance_id": match.group(4),
             "event_time": next(log_file).split("start")[1].split("\n")[0],
             "serial_number": match.group(6)
         })

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(usb_devices)

# Display the tail of the DataFrame
df.tail(20)

,device_vendor_id,device_product_id,device_instance_id,event_time,serial_number
0,Disk&Ven_Verbatim,Prod_STORE_N_GO,Rev_1100,2022/11/03 15:00:17.389,1311130000000480&0
1,Disk&Ven_SMI,Prod_USB_DISK,Rev_1100,2023/02/01 11:10:46.945,6&7d7590d&0
2,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,2023/02/06 12:50:42.455,920707402A3F8066956&0
3,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,2023/02/06 12:51:00.660,9207113E5CF83172266&0
4,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,2023/02/06 12:52:25.253,AC220B280549EF6059EEBAFE&0
5,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,2023/03/23 08:13:57.618,AC220B280549EF6059EEBAFE&0
6,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,2023/03/29 16:28:31.704,6&22ff249f&1
7,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,2023/03/29 16:32:50.762,6&104573fa&1


In [29]:

import os
import hashlib
import pandas as pd
import time

extensions = []
sha256s = []
sha1s = []
md5s = []
creation_times = []
modification_times = []
access_times = []

def file2hash(filename, hash_function):
    with open(filename, 'rb') as f:
        return hash_function(f.read()).hexdigest()

dir_path = 'E:\\'

for file in os.listdir(dir_path):
    path = os.path.join(dir_path, file)
    if os.path.isfile(path):
        sha256 = file2hash(path, hashlib.sha256)
        sha256s.append(sha256)

        sha1 = file2hash(path, hashlib.sha1)
        sha1s.append(sha1)

        md5 = file2hash(path, hashlib.md5)
        md5s.append(md5)

        _, extension = os.path.splitext(file)
        extensions.append(extension)
        
        creation_time = os.path.getctime(path)
        modification_time = os.path.getmtime(path)
        access_time = os.path.getatime(path)
        
        creation_times.append(time.ctime(creation_time))
        modification_times.append(time.ctime(modification_time))
        access_times.append(time.ctime(access_time))


file_names = []
for file in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, file)):
        file_names.append(file)

df2 = pd.DataFrame({
    "file_name": file_names,
    "extension": extensions,
    "sha256s": sha256s,
    "md5": md5s,
    "sha1": sha1s,
    "creation_times": creation_times,
    "modification_times": modification_times,
    "access_times:": access_times
})

df2.tail(20)

,file_name,extension,sha256s,md5,sha1,creation_times,modification_times,access_times:
0,Secret_file_11.txt,.txt,5f90ecf090772e3366353171a2e8b9c4001934cdc3a5e0...,187b0341b64a0e16d74ea3b50e22e6c2,08152ff79a16f2a181174eb5fe8de0197f4d722f,Wed Mar 29 17:35:08 2023,Mon Oct 28 16:08:46 2019,Wed Mar 29 01:00:00 2023
1,Secret_file_92.pdf,.pdf,3e174794698cbdf84d9a5cd352f755d0e6bdfc5dc65e0d...,4266ea20320c598bc5d7b1d731e3e9e9,1a30a1d139131e7105a1896d7719c85c03003a59,Wed Mar 29 17:35:10 2023,Mon Oct 28 16:09:32 2019,Wed Mar 29 01:00:00 2023
